In [7]:
#This part of code, the goal is to go from mapping and county level weather data to to subgroup data
# Define paths
data_and_output_dir = '/Users/ansonkong/Downloads/Data for nyu work/'
full_future_data_path = '/Users/ansonkong/Downloads/rcp85hotter/'
full_historical_data_path = '/Users/ansonkong/Downloads/historic/'

#this just smooth out operation
cutoff_year=2020 #The year prediction dataset start
start_year=2000 #The year historical 
end_year=2100

#Debug mode(only output once)
debug=False

In [53]:
#import require file
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
import os
from datetime import datetime
from glob import glob

ba_dir = data_and_output_dir+'Input/EIA 930/BA/'


# Function to read and merge CSVs from a directory
def read_and_merge_csvs(directory, cols):
    combined_df = pd.DataFrame()
    for file in os.listdir(directory):
        if file.endswith('.csv'):
            df = pd.read_csv(os.path.join(directory, file), usecols=cols)
            combined_df = pd.concat([combined_df, df], ignore_index=True)
    return combined_df

ba_cols = ['UTC Time at End of Hour', 'Balancing Authority', 'Demand (MW)']
ba_combined_df = read_and_merge_csvs(ba_dir, ba_cols)



#Path, this will update itself
mapping_path = data_and_output_dir + 'output/merged_rb_control_area_mapping.csv'
future_population_data_path = data_and_output_dir + 'input/Electric_Retail_Service_Territories/ssp3_county_population.csv'
historical_population_data_path = data_and_output_dir + 'input/county_populations_2000_to_2020.csv'
output_path = data_and_output_dir + 'output/'

# Load mapping and population data
mapping_df = pd.read_csv(mapping_path)
future_population_df = pd.read_csv(future_population_data_path)
historical_population_df = pd.read_csv(historical_population_data_path)

#crate looping for interp
years = np.arange(cutoff_year, end_year + 1, 10)
interp_years = np.arange(cutoff_year, end_year + 1)

# Ensure FIPS codes are five zero-padded strings
historical_population_df['FIPS'] = historical_population_df['county_FIPS'].astype(str).str.zfill(5)
future_population_df['FIPS'] = future_population_df['FIPS'].astype(str).str.zfill(5)
mapping_df['GEOID'] = mapping_df['GEOID'].astype(str).str.zfill(5)

# Rename 'pop_{year}' columns to just '{year}'
for year in range(start_year, cutoff_year+1):
    if f'pop_{year}' in historical_population_df.columns:
        historical_population_df.rename(columns={f'pop_{year}': str(year)}, inplace=True)



# Assuming 'FIPS' is the column to join on and 'Year' is the column indicating the year in future_population_df
for index, row in future_population_df.iterrows():
    pop_values = [row[str(year)] for year in years if str(year) in row]
    interpolator = interp1d(years, pop_values, kind='linear', fill_value="extrapolate")
    interpolated_values = interpolator(interp_years)
    for year in interp_years:
        future_population_df.at[index, str(year)] = interpolated_values[year - cutoff_year]


# Select relevant columns for historical data
historical_population_df = historical_population_df[['FIPS'] + [str(year) for year in range(start_year, cutoff_year)]]

# Select relevant columns for future data
future_population_df = future_population_df[['FIPS'] + [str(year) for year in interp_years]]

# Concatenate historical and future dataframes
combined_population_df = pd.merge(historical_population_df, future_population_df, on='FIPS', how='outer')
# Merge combined population data with mapping data
# Assuming 'GEOID' in mapping_df corresponds to 'FIPS' in population data
combined_df = pd.merge(mapping_df, combined_population_df, left_on='GEOID', right_on='FIPS', how='inner')
combined_df['FIPS']=combined_df['GEOID']

/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_7172/2308327632.py:17: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(directory, file), usecols=cols)
/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_7172/2308327632.py:17: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(directory, file), usecols=cols)
/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_7172/2308327632.py:17: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(directory, file), usecols=cols)
/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_7172/2308327632.py:17: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(directory, file), usecols=cols)
/var/folders/8v/

In [54]:
combined_df['FIPS']=combined_df['FIPS_x']
combined_df=combined_df[['FIPS','rb','geometry']+[str(year) for year in range(start_year, end_year+1)]]

In [72]:
# Generate the expected set of 'rb' values ('p1' to 'p134')
expected_rbs = {'p' + str(i) for i in range(1, 135)}

# Extract the unique 'rb' values from combined_df
actual_rbs = set(combined_df['rb'].unique())

# Check if combined_df['rb'] contains all values from 'p1' to 'p134'
missing_rbs = expected_rbs - actual_rbs
extra_rbs = actual_rbs - expected_rbs

# Display the results
if not missing_rbs:
    print("All 'rb' values from 'p1' to 'p134' are present in combined_df.")
else:
    print(f"Missing 'rb' values in combined_df: {missing_rbs}")

if extra_rbs:
    print(f"Extra 'rb' values in combined_df not in the range 'p1' to 'p134': {extra_rbs}")


All 'rb' values from 'p1' to 'p134' are present in combined_df.


In [58]:
rb_to_ba=pd.read_csv(data_and_output_dir + 'output/rb_to_ba.csv')
rb_to_ba

,rb,geometry,BA_Code
0,p1,MULTIPOLYGON (((-13687924.23979998 5965691.031...,BPAT
1,p10,MULTIPOLYGON (((-13179969.92729995 3871459.308...,CISO
2,p100,MULTIPOLYGON (((-8663603.907299994 4741636.251...,PJM
3,p101,MULTIPOLYGON (((-9090644.388599997 2824968.523...,SEC
4,p102,MULTIPOLYGON (((-8924334.972400004 2980193.438...,FPL
...,...,...,...
129,p95,MULTIPOLYGON (((-9114960.995699998 4188937.632...,DUK
130,p96,MULTIPOLYGON (((-8956515.669899996 3812869.837...,SC
131,p97,MULTIPOLYGON (((-8873816.772200003 4375448.743...,DUK
132,p98,MULTIPOLYGON (((-8678230.449399997 4014627.318...,CPLE


In [59]:
rb_to_ba['BA_Code'].unique()

array(['BPAT', 'CISO', 'PJM', 'SEC', 'FPL', 'MISO', 'NEVP', 'NYIS',
       'ISNE', 'WALC', 'AVA', 'IPCO', 'PACE', 'NWMT', 'SWPP', 'WACM',
       'AZPS', 'PNM', 'PSCO', 'AECI', 'EPE', 'PACW', 'ERCO', 'TVA',
       'SOCO', 'AEC', 'DUK', 'SC', 'CPLE'], dtype=object)

In [73]:
population_with_county_and_rb_df = pd.merge(combined_df, rb_to_ba, on='rb', how='left')

In [74]:
# Extract the unique 'rb' values from combined_df
actual_rbs = set(population_with_county_and_rb_df ['rb'].unique())

# Check if combined_df['rb'] contains all values from 'p1' to 'p134'
missing_rbs = expected_rbs - actual_rbs
extra_rbs = actual_rbs - expected_rbs

# Display the results
if not missing_rbs:
    print("All 'rb' values from 'p1' to 'p134' are present in combined_df.")
else:
    print(f"Missing 'rb' values in combined_df: {missing_rbs}")

if extra_rbs:
    print(f"Extra 'rb' values in combined_df not in the range 'p1' to 'p134': {extra_rbs}")


All 'rb' values from 'p1' to 'p134' are present in combined_df.


In [66]:
# Create a list of the year columns to sum up
year_columns = [str(year) for year in range(start_year, end_year + 1)]

# Group by 'rb' and 'BA_Code', then sum up the specified year columns
aggregated_df = population_with_county_and_rb_df.groupby(['rb', 'BA_Code'])[year_columns].sum().reset_index()


In [67]:
# Loop through each year column to perform the transformation
for year in year_columns:
    # Calculate the sum for each 'rb' group in the current year column
    rb_sum = aggregated_df.groupby('BA_Code')[year].transform('sum')
    
    # Divide each value by its corresponding 'rb' total for the current year
    aggregated_df[year] = aggregated_df[year] / rb_sum

# Display the first few rows of the transformed DataFrame to verify the results
print(aggregated_df.head())


     rb BA_Code      2000      2001      2002      2003      2004      2005  \
0    p1    BPAT  0.450298  0.450676  0.449738  0.448958  0.448836  0.448224   
1   p10    CISO  0.520191  0.520019  0.521367  0.522828  0.524362  0.524909   
2  p100     PJM  0.003103  0.003143  0.003189  0.003227  0.003270  0.003331   
3  p101     SEC  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000   
4  p102     FPL  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000   

       2006      2007  ...      2091      2092      2093      2094      2095  \
0  0.448384  0.447617  ...  0.455866  0.456058  0.456253  0.456450  0.456651   
1  0.524787  0.523626  ...  0.553351  0.553587  0.553828  0.554073  0.554323   
2  0.003387  0.003424  ...  0.003885  0.003872  0.003860  0.003847  0.003834   
3  1.000000  1.000000  ...  1.000000  1.000000  1.000000  1.000000  1.000000   
4  1.000000  1.000000  ...  1.000000  1.000000  1.000000  1.000000  1.000000   

       2096      2097      2098      2099   

In [75]:
# Extract the unique 'rb' values from combined_df
actual_rbs = set(aggregated_df['rb'].unique())

# Check if combined_df['rb'] contains all values from 'p1' to 'p134'
missing_rbs = expected_rbs - actual_rbs
extra_rbs = actual_rbs - expected_rbs

# Display the results
if not missing_rbs:
    print("All 'rb' values from 'p1' to 'p134' are present in combined_df.")
else:
    print(f"Missing 'rb' values in combined_df: {missing_rbs}")

if extra_rbs:
    print(f"Extra 'rb' values in combined_df not in the range 'p1' to 'p134': {extra_rbs}")

All 'rb' values from 'p1' to 'p134' are present in combined_df.


In [78]:
ba_combined_df

,Balancing Authority,UTC Time at End of Hour,Demand (MW)
0,AEC,01/01/2018 7:00:00 AM,748
1,AEC,01/01/2018 8:00:00 AM,763
2,AEC,01/01/2018 9:00:00 AM,789
3,AEC,01/01/2018 10:00:00 AM,814
4,AEC,01/01/2018 11:00:00 AM,848
...,...,...,...
3158641,YAD,01/01/2019 1:00:00 AM,NaN
3158642,YAD,01/01/2019 2:00:00 AM,NaN
3158643,YAD,01/01/2019 3:00:00 AM,NaN
3158644,YAD,01/01/2019 4:00:00 AM,NaN


In [81]:
# Assuming ba_combined_df and rb_to_ba are your DataFrames

# Extract unique 'Balancing Authority' and 'BA_Code' values
unique_ba_combined = set(ba_combined_df['Balancing Authority'].unique())
unique_rb_to_ba = set(rb_to_ba['BA_Code'].unique())

# Find codes that are unique to ba_combined_df
unique_to_ba_combined = unique_ba_combined - unique_rb_to_ba

# Find codes that are unique to rb_to_ba
unique_to_rb_to_ba = unique_rb_to_ba - unique_ba_combined

# Find codes that are present in both
perfect_matches = unique_ba_combined.intersection(unique_rb_to_ba)

# Display the results
print(f"Codes unique to ba_combined_df: {unique_to_ba_combined}")
print(f"Codes unique to rb_to_ba: {unique_to_rb_to_ba}")
print(f"Perfect matches: {perfect_matches}")


Codes unique to ba_combined_df: {'AVRN', 'SPA', 'GLHB', 'NSB', 'FMPP', 'HGMA', 'GRID', 'GRMA', 'DOPD', 'GVL', 'WWA', 'TAL', 'WAUW', 'IID', 'DEAA', 'SCL', 'TEPC', 'LGEE', 'GWA', 'SEPA', 'YAD', 'GCPD', 'EEI', 'JEA', 'PSEI', 'CPLW', 'FPC', 'SRP', 'LDWP', 'TIDC', 'OVEC', 'WAUE', 'CHPD', 'TEC', 'BANC', 'PGE', 'SCEG', 'TPWR', 'GRIF', 'HST'}
Codes unique to rb_to_ba: set()
Perfect matches: {'SWPP', 'SOCO', 'SC', 'PNM', 'FPL', 'IPCO', 'AEC', 'CPLE', 'AZPS', 'ERCO', 'MISO', 'PJM', 'TVA', 'BPAT', 'NWMT', 'EPE', 'NYIS', 'PACW', 'AVA', 'WALC', 'CISO', 'AECI', 'DUK', 'WACM', 'PACE', 'SEC', 'ISNE', 'NEVP', 'PSCO'}


In [85]:
csv_path = data_and_output_dir + 'Input/ba_service_territory_2020.csv'
# Read the CSV file
df = pd.read_csv(csv_path)
df['BA_Code'].unique()

array(['NBSO', 'AEC', 'AMPL', 'AZPS', 'AECI', 'BPAT', 'CISO', 'CPLE',
       'CHPD', 'CEA', 'DOPD', 'DUK', 'EPE', 'ERCO', 'FPL', 'FPC', 'GVL',
       'HST', 'IPCO', 'IID', 'JEA', 'LDWP', 'LGEE', 'NWMT', 'NEVP',
       'ISNE', 'NSB', 'NYIS', 'OVEC', 'PACW', 'PACE', 'FMPP', 'GCPD',
       'PJM', 'PGE', 'PSCO', 'PNM', 'PSEI', 'BANC', 'SRP', 'SCL', 'SCEG',
       'SC', 'SPA', 'SOCO', 'TPWR', 'TAL', 'TEC', 'TVA', 'TIDC', 'HECO',
       'WAUW', 'AVA', 'SEC', 'TEPC', 'WALC', 'WACM', 'SEPA', 'MISO',
       'SWPP'], dtype=object)